In [1]:
import json
data_dir = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_clipped_processed/"+'melody.json'
queries = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_queries_processed/"+'melody.json'


In [2]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [3]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [4]:
from mongeau_sankoff import similarity, Note
import miditoolkit as mtk
# queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)

In [5]:
obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")

In [6]:
notes = obj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

ans = 99999
prev_end = 0
for n in notes:
    if n.start > prev_end:
        dur = n.start - prev_end
        ans = min(ans, dur)
    ans = min(ans, n.end - n.start)
    prev_end = n.end
t= ans
        


In [7]:
rep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        rep.append(Note("C",int((n.start-prev_end)/t),rest=True))
    rep.append(Note(midiEt_to_note[n.pitch%12 + 12],int((n.end-n.start)/t),rest=False))
    prev_end = n.end

In [8]:
#qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")
qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_query_melody.mid")


In [9]:
notes = qobj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

ans = 99999
prev_end = 0
for n in notes:
    if n.start > prev_end:
        dur = n.start - prev_end
        ans = min(ans, dur)
    ans = min(ans, n.end - n.start)
    prev_end = n.end
t = ans


In [10]:
qrep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        qrep.append(Note("C", int((n.start-prev_end)/t), rest=True))
    qrep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False))
    prev_end = n.end


In [11]:
similarity(qrep, rep)

75it [00:00, 191.82it/s]


-0.015000000000000346

In [12]:
from tqdm import tqdm
import os
import music21
fnames = os.listdir("C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/")


def getKey(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.tonic.name)
def getMode(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.mode)


In [19]:
def compute_similarities(qrep):
    fname_to_similarity = {}
    for fname in fnames:
        sim = 0
        obj = mtk.MidiFile(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        notes = obj.instruments[0].notes 
        ans = 99999
        prev_end = 0
        
        for n in notes:
            if n.start > prev_end:
                dur = n.start - prev_end
                ans = min(ans, dur)
            ans = min(ans, n.end - n.start)
            prev_end = n.end
        t = ans
        mode = getMode(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        
        rep = []
        prev_end = 0
        for n in notes:
            if n.start > prev_end:
                rep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
            rep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
            prev_end = n.end
        try:
            sim = similarity(qrep, rep)
            fname_to_similarity[fname] = sim
        except:
            continue
    return fname_to_similarity


In [20]:
fname_to_similarity = compute_similarities(qrep)

75it [00:00, 127.12it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 117.71it/s]              


13it [00:00, 130.00it/s]             
65it [00:00, 149.08it/s]
100%|██████████| 5/5 [00:00<00:00, 142.90it/s]
68it [00:00, 152.47it/s]
15it [00:00, 132.74it/s]
18it [00:00, 131.39it/s]
42it [00:00, 139.07it/s]
117it [00:00, 138.63it/s]
38it [00:00, 146.71it/s]
 89%|████████▉ | 8/9 [00:00<00:00, 140.40it/s]
58it [00:00, 142.17it/s]
51it [00:00, 119.72it/s]
18it [00:00, 113.21it/s]                        
41it [00:00, 121.30it/s]
93it [00:00, 124.67it/s]
29it [00:00, 113.73it/s]
100it [00:00, 138.12it/s]
39it [00:00, 151.17it/s]
79it [00:00, 142.09it/s]
 67%|██████▋   | 10/15 [00:00<00:00, 129.86it/s]
74it [00:00, 164.08it/s]
86it [00:00, 157.22it/s]
41it [00:00, 159.53it/s]
34it [00:00, 163.45it/s]


51it [00:00, 163.93it/s]



 30%|███       | 3/10 [00:00<00:00, 142.84it/s]








20it [00:00, 106.38it/s]




















49it [00:00, 108.17it/s]












32it [00:00, 128.52it/

In [21]:
dict(sorted(fname_to_similarity.items(), key=lambda item: item[1], reverse=True))

{'MIDI-Unprocessed_Schubert10-12_MID--AUDIO_17_R2_2018_wav_melody.mid': 0.5650000000000002,
 'MIDI-Unprocessed_Schubert7-9_MID--AUDIO_15_R2_2018_wav_melody.mid': 0.5070000000000001,
 'MIDI-Unprocessed_Schubert10-12_MID--AUDIO_18_R2_2018_wav_melody.mid': 0.3910000000000001,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_18_R1_2018_wav--3_melody.mid': 0.246,
 'MIDI-Unprocessed_Schubert4-6_MID--AUDIO_08_R2_2018_wav_melody.mid': 0.21699999999999997,
 'MIDI-Unprocessed_Recital5-7_MID--AUDIO_05_R1_2018_wav--2_melody.mid': 0.18799999999999983,
 'MIDI-Unprocessed_Recital9-11_MID--AUDIO_09_R1_2018_wav--4_melody.mid': 0.18799999999999983,
 'MIDI-Unprocessed_Recital9-11_MID--AUDIO_09_R1_2018_wav--6_melody.mid': 0.18799999999999983,
 'MIDI-Unprocessed_Recital4_MID--AUDIO_04_R1_2018_wav--4_melody.mid': 0.10099999999999987,
 'MIDI-Unprocessed_Recital5-7_MID--AUDIO_06_R1_2018_wav--2_melody.mid': 0.10099999999999987,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_17_R1_2018_wav--2_melody.mid': 0.0719999999999